In [1]:
import pandas as pd
import numpy as np
import requests 
import itertools

In [2]:
#read the data file which contains features of the objects in the Hackathon galleries
features = pd.read_csv("limited_objectinfo.csv", encoding = 'latin-1') 

In [3]:
#create a list of the object ids of the ten works which the user is shown.
tenworks = pd.read_excel("tenworks.xlsx")
obj = list(tenworks.ObjectID)

In [7]:
def featmat(obj, pref) : 
    feat = pd.read_csv("limited_objectinfo.csv", encoding='latin-1')
    feat.drop(["Geography"], axis = 1, inplace = True)
    feat = pd.get_dummies(feat, columns = ["Classification", "Style"]) 
    
    #reindex according to ObjectID
    df = feat.set_index("ObjectID")
    df.fillna(0, inplace = True) 
    df.drop(["Unnamed: 0"], axis = 1, inplace = True)
   
    #feature_matrix will contain only the ten rows corresponding to the art shown to the user
    feature_matrix = df.loc[obj]

    #feature_matrix.drop(["Unnamed: 0"], axis = 1, inplace = True)

    adapted_matrix = np.zeros((10,np.shape(feature_matrix)[1] )) 

    for i in range(len(pref)) : 
        adapted_matrix[i] = pref[i] * feature_matrix.iloc[i]

    #for each column, find the mean
    feature_means = adapted_matrix.mean(axis = 0) 
 
    norms = []
    for ind in df.index : 
        norms.append({"objid":ind, "norm":np.linalg.norm(np.array(df.loc[ind])-np.array(feature_means), ord = 1)})
    norms_df = pd.DataFrame(norms)
    norms_df.sort_values("norm", axis =0, inplace = True)

    normalized = [1-1/(1+norms_df.norm.iloc[i]) for i in range(len(norms_df))]
    norms_df["normalized"] = normalized 
    norms_df.sort_values("normalized", axis = 0, ascending = False, inplace = True) 

    return(norms_df) 
    

In [5]:
pref = [1,1,-1,-1,-1,1,-1,-1, -1, 1] 
k = featmat(obj, pref)

In [ ]:
frames = [] 
for ind in range(1024) : 
    top3 = list(featmat(obj, l[ind]).objid ) 
    rating_dict = {"index":ind, "value":top3}
    frames.append(rating_dict) 
    if ind%100 == 0 : 
           print(ind) 